In [35]:
import polars as pl
import plotly.express as px
import plotly.io as pio
pl.Config(tbl_rows = -1, tbl_cols = -1, fmt_str_lengths = 1000, tbl_width_chars = 1000)
pio.templates.default = "plotly_white"

In [36]:
# read the csv data into a polars dataframe
df = pl.read_csv("1976-2020-president.csv", ignore_errors = True)

In [37]:
# some candidates appear under multiple parties
# for example, Clinton, Johnson, and Trump all appear under multiple parties in New York in 2016
print(
    df.filter((pl.col("year") == 2016) & (pl.col("state_po") == "NY")). \
        select([pl.col(col) for col in ["year", "state", "state_po", "candidate", "candidatevotes", "party_simplified", "party_detailed"]]). \
        sort("candidate")
)

shape: (12, 7)
┌──────┬──────────┬──────────┬──────────────────┬────────────────┬──────────────────┬──────────────────┐
│ year ┆ state    ┆ state_po ┆ candidate        ┆ candidatevotes ┆ party_simplified ┆ party_detailed   │
│ ---  ┆ ---      ┆ ---      ┆ ---              ┆ ---            ┆ ---              ┆ ---              │
│ i64  ┆ str      ┆ str      ┆ str              ┆ i64            ┆ str              ┆ str              │
╞══════╪══════════╪══════════╪══════════════════╪════════════════╪══════════════════╪══════════════════╡
│ 2016 ┆ NEW YORK ┆ NY       ┆                  ┆ 12816          ┆ OTHER            ┆                  │
│ 2016 ┆ NEW YORK ┆ NY       ┆ BLANK VOTE       ┆ 77179          ┆ OTHER            ┆                  │
│ 2016 ┆ NEW YORK ┆ NY       ┆ CLINTON, HILLARY ┆ 4379789        ┆ DEMOCRAT         ┆ DEMOCRAT         │
│ 2016 ┆ NEW YORK ┆ NY       ┆ CLINTON, HILLARY ┆ 140041         ┆ OTHER            ┆ WORKING FAMILIES │
│ 2016 ┆ NEW YORK ┆ NY       ┆ CLINTON, 

In [38]:
# clean the data 
# group by year, state, and candidate 
# find the sum of votes received by each candidate for each year and state
# find the total number of votes cast for all candidates for each year and state
# find the party of each candidate using case/when statements because candidates can appear under multiple parties
df = df.group_by(["year", "state", "state_po", "candidate"]).agg([
    pl.col("candidatevotes").sum().alias("candidatevotes"),
    pl.col("totalvotes").max().alias("totalvotes"),
    (pl.col("candidatevotes").sum() / pl.col("totalvotes").max()).alias("candidatevotes_prop"),
    pl.when(
        (pl.col("party_simplified") == "REPUBLICAN").sum() > 0
    ).then(pl.lit("REPUBLICAN")).when(
        (pl.col("party_simplified") == "DEMOCRAT").sum() > 0
    ).then(pl.lit("DEMOCRAT")).otherwise(pl.lit("OTHER")).alias("party_simplified")
])

In [39]:
group_var = ["year", "state", "state_po"]
state_group_var = ["state", "state_po"]
# find the proportion of libertarian or other votes for each state and year
other_party_prop_df = df.filter(pl.col("party_simplified") == "OTHER"). \
    group_by(group_var). \
    agg(pl.col("candidatevotes_prop").sum().alias("candidatevotes_prop")).\
    sort(group_var)
# find the proportion of republican votes (compared to democrat votes) for each state and year
rep_party_prop_df = df.join(other = df.filter(pl.col("party_simplified") != "OTHER").group_by(group_var).agg(pl.col("candidatevotes").sum().alias("totalvotes_no_other")), on = group_var). \
    filter(pl.col("party_simplified") == "REPUBLICAN"). \
    with_columns((pl.col("candidatevotes") / pl.col("totalvotes_no_other")).alias("candidatevotes_prop_no_other")). \
    sort(group_var)
# find the difference between the maximum and minimum proportions of republican votes for each state (over all years)
rep_party_prop_diff_df = rep_party_prop_df.group_by(state_group_var). \
    agg(pl.col("candidatevotes_prop").min().alias("prop_min")). \
    join(rep_party_prop_df.group_by(state_group_var).agg(pl.col("candidatevotes_prop").max().alias("prop_max")), on = state_group_var). \
    with_columns((pl.col("prop_max") - pl.col("prop_min")).alias("candidatevotes_prop_diff")). \
    sort("candidatevotes_prop_diff", descending = True)
# find the states with the largest differences between the maxmimum and minimum proportions of republican votes
top_diff_states = list(rep_party_prop_diff_df.top_k(10, by = "candidatevotes_prop_diff")["state_po"])
# find the third party candidates that received at least 3% of the vote
third_party_df = df.filter((pl.col("party_simplified") == "OTHER") & (pl.col("candidatevotes_prop") >= 0.03) & ~(pl.col("candidate").is_in(["", "OTHER", "NONE OF THE ABOVE", "BLANK VOTE", "NOT DESIGNATED"])))

In [40]:
px.choropleth(
    data_frame = rep_party_prop_df,
    geojson="https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json",
    locationmode = "USA-states",
    locations = "state_po",
    color = "candidatevotes_prop_no_other",
    scope = "usa",
    color_continuous_midpoint = 0.5,
    color_continuous_scale = [(0, "blue"), (0.3, "blue"), (0.5, "purple"), (0.7, "red"), (1, "red")],
    animation_frame = "year",
    title = "<b>Proportion of Votes for Republican Candidates (Compared to Democrat Candidates) by State and Election Year</b><br><i>US Presidential Elections 1976 - 2020</i>",
    labels = {"candidatevotes_prop_no_other": "Proportion Republican", "year": "Election Year", "state_po": "State"},
    height = 600
)

C:\Users\lesch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [41]:
px.line(
    data_frame = rep_party_prop_df.filter(pl.col("state_po").is_in(top_diff_states)),
    x = "year", 
    y = "candidatevotes_prop",
    title = "<b>Proportion of Votes for Republican Candidates (Compared to Democrat Candidates) by State and Election Year</b><br><i>US Presidential Elections 1976 - 2020, States with Top 10 Shifts</i>",
    labels = {"candidatevotes_prop": "Proportion Republican", "year": "Election Year", "state": "State"},
    color = "state",
    color_discrete_sequence = px.colors.qualitative.Prism,
    markers = True,
    height = 600
)

C:\Users\lesch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [42]:
px.box(
    data_frame = other_party_prop_df,
    x = "year", 
    y = "candidatevotes_prop",
    color_discrete_sequence = px.colors.qualitative.Prism,
    title = "<b>Distribution of Proportion of Votes for Third Party Candidates by Election Year</b><br><i>US Presidential Elections 1976 - 2020</i>",
    labels = {"candidatevotes_prop": "Proportion Third Party", "year": "Election Year"},
    height = 600
)

In [43]:
px.bar(
    data_frame = third_party_df. \
        group_by(["year", "candidate"]).len(). \
        sort(["year", "candidate"]). \
        with_columns(pl.col("year").cast(pl.Utf8)), 
    x = "year", 
    y = "len", 
    color = "candidate", 
    text = "candidate",
    title = "<b>Third Party Candidates That Received At Least 3% of Vote by Election Year</b><br><i>US Presidential Elections 1976 - 2020</i>",
    labels = {"len": "Number of States", "year": "Election Year", "candidate": "Candidate"},
    color_discrete_sequence = px.colors.qualitative.Prism,
    height = 600,
).update_layout(uniformtext_minsize = 10, uniformtext_mode = "hide")

C:\Users\lesch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [44]:
px.choropleth(
    data_frame = third_party_df. \
        group_by(["year", "state", "state_po"]).len(). \
        group_by(["state", "state_po"]).len(). \
        sort("len"). \
        with_columns(pl.col("len").cast(pl.Float64)),
    geojson = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json",
    locationmode = "USA-states",
    locations = "state_po",
    color = "len",
    color_continuous_scale = px.colors.sequential.Blackbody[::-1],
    scope = "usa",
    height = 600,
    title = "<b>Number of Elections Where Third Party Candidate Received At Least 3% of Vote</b><br><i>US Presidential Elections 1976 - 2020</i>"
)